In [1]:
# mkdir if not exist 'proposals_out_2023_07_21'
import os
if not os.path.exists('proposals_out_2023_07_21'):
    os.makedirs('proposals_out_2023_07_21')

In [2]:
# read in ../../DATA/deployments/snapshot_deployments.csv
import pandas as pd
df = pd.read_csv('../../DATA/deployments/snapshot_deployments.csv')
space_ids = df.platform_id.unique()
len(space_ids)

26902

In [3]:
df.head()

,platform,platform_id,name,additional,website,votes_count,proposals_count
0,snapshot,ethfund.eth,ETHFUND,NaN,https://snapshot.org/#/ethfund.eth,1,1
1,snapshot,fadedfate.eth,Magic Girls,liuhaha5308848,https://snapshot.org/#/fadedfate.eth,1,3
2,snapshot,fangnaoke.eth,study group,https://app.syndicate.io/collectives/0x5f51845...,https://snapshot.org/#/fangnaoke.eth,1,1
3,snapshot,fasola.eth,FASOLA.ETH,NaN,https://snapshot.org/#/fasola.eth,7,1
4,snapshot,fdsfsfss.eth,alex,NaN,https://snapshot.org/#/fdsfsfss.eth,3,3


In [4]:
df.proposals_count.describe()

count    27471.000000
mean         4.371883
std         30.036288
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max       3177.000000
Name: proposals_count, dtype: float64

In [5]:
# remove the space_ids that have no proposals
df = df[df.proposals_count > 0]
platforms_and_proposal_count = df[['platform_id', 'proposals_count']].sort_values(by='proposals_count', ascending=False)
# make these into a list of tuples
platforms_and_proposal_count = list(platforms_and_proposal_count.itertuples(index=False, name=None))
platforms_and_proposal_count[:5]

[('cakevote.eth', 3177),
 ('snapshot.dcl.eth', 2226),
 ('index-coop.eth', 1052),
 ('frami.eth', 657),
 ('huwacoin.eth', 631)]

In [6]:
# make groups of rows where the proposals_count in the group sums to less than 5000
groups_of_space_ids_that_sum_to_less_than_5000 = []
while len(platforms_and_proposal_count) > 0:
    group = []
    total = 0
    while total < 5000:
        if platforms_and_proposal_count[0][1] + total > 5000:
            break
        row = platforms_and_proposal_count.pop(0)
        group.append(row)
        total += row[1]
        if len(platforms_and_proposal_count) == 0:
            break
    groups_of_space_ids_that_sum_to_less_than_5000.append(group)

groups_of_space_ids_that_sum_to_less_than_5000

[[('cakevote.eth', 3177)],
 [('snapshot.dcl.eth', 2226),
  ('index-coop.eth', 1052),
  ('frami.eth', 657),
  ('huwacoin.eth', 631)],
 [('balancer.eth', 602),
  ('bentfinance.eth', 542),
  ('bancornetwork.eth', 541),
  ('sdbal.eth', 535),
  ('lemu.dcl.eth', 476),
  ('sharkdao.eth', 471),
  ('aave.eth', 445),
  ('aavegotchi.eth', 433),
  ('aurafinance.eth', 415),
  ('fabien.eth', 367)],
 [('orbapp.eth', 365),
  ('frax.eth', 359),
  ('alluo.eth', 357),
  ('jbdao.eth', 354),
  ('goodmorningnews.eth', 352),
  ('aladdindao.eth', 331),
  ('magicappstore.eth', 319),
  ('fuse.eth', 317),
  ('goopsnapshot.eth', 315),
  ('leagueoflils.eth', 299),
  ('sdfxs.eth', 289),
  ('olympusdao.eth', 272),
  ('狂奔的蜗牛1.eth', 271),
  ('17707.eth', 262),
  ('primerating.eth', 261),
  ('veclev.eth', 260)],
 [('xdaistake.eth', 257),
  ('bgansv2.eth', 255),
  ('fantomecosystem.eth', 250),
  ('decentralgames.eth', 238),
  ('lido-snapshot.eth', 228),
  ('karastar.eth', 222),
  ('theheaddao.eth', 219),
  ('pnounsdao.e

In [7]:
len(groups_of_space_ids_that_sum_to_less_than_5000)

25

In [8]:
# sum the proposals_count in each group
sums = [sum([x[1] for x in group]) for group in groups_of_space_ids_that_sum_to_less_than_5000]
sums

[3177,
 4566,
 4827,
 4983,
 4918,
 4910,
 4967,
 4951,
 4943,
 4992,
 4967,
 4977,
 4984,
 4998,
 4987,
 4997,
 5000,
 4994,
 4997,
 4998,
 4999,
 5000,
 5000,
 5000,
 2968]

In [9]:
# make into a list of strings for the graphql query
groups_of_space_ids_that_sum_to_less_than_5000_strings = []
for group in groups_of_space_ids_that_sum_to_less_than_5000:
    group_of_space_ids = [str(x[0]) for x in group]
    query_string = ", ".join([f'"{sid}"' for sid in group_of_space_ids])
    groups_of_space_ids_that_sum_to_less_than_5000_strings.append(query_string)

groups_of_space_ids_that_sum_to_less_than_5000_strings[0]

'"cakevote.eth"'

In [10]:
def make_query(space_ids: str, skip: int) -> str:
    first = 1000
    base = """
        query Proposals {
         proposals(where: {space_in: [%s], state: "closed"}, first: %s, skip: %s, orderDirection: desc, orderBy: "created") {
            id
            space {
              id
            }
            author
            created
            votes
            title
            body
            start
            end
          }
        }
    """
    return base % (space_ids, first, skip)


In [11]:
import requests
import backoff
URL = "https://hub.snapshot.org/graphql"


@backoff.on_exception(backoff.expo,
                      requests.exceptions.RequestException)
def query_snapshot(query):
    r = requests.post(URL, json={'query': query})
    return r.json()

In [12]:
import json
for count, space_id_group in enumerate(groups_of_space_ids_that_sum_to_less_than_5000_strings):
    # make pagination logic
    skip = 0
    has_more = True
    while has_more:
        filename = f'proposals_out_2023_07_21/proposals_{count}_{skip}.json'
        next_proposals_filename = f'proposals_out_2023_07_21/proposals_{count}_{skip + 1000}.json'
        if os.path.exists(next_proposals_filename):
            print(f'next file exists: {next_proposals_filename}')
            skip += 1000
            continue
        next_group_filename = f'proposals_out_2023_07_21/proposals_{count + 1}_0.json'
        if os.path.exists(next_group_filename):
            print(f'next group file exists: {next_group_filename}')
            has_more = False
            continue
        query = make_query(space_id_group, skip)
        result = query_snapshot(query)
        proposals = result['data']['proposals']
        print('got', len(proposals), 'proposals for', count, skip)
        # save proposals to file in the output dir
        with open(filename, 'w') as f:
            json.dump(proposals, f)
        if len(proposals) == 1000:
            skip += 1000
        else:
            has_more = False


next file exists: proposals_out_2023_07_21/proposals_0_1000.json
next file exists: proposals_out_2023_07_21/proposals_0_2000.json
next file exists: proposals_out_2023_07_21/proposals_0_3000.json
next group file exists: proposals_out_2023_07_21/proposals_1_0.json
next file exists: proposals_out_2023_07_21/proposals_1_1000.json
next file exists: proposals_out_2023_07_21/proposals_1_2000.json
next file exists: proposals_out_2023_07_21/proposals_1_3000.json
next file exists: proposals_out_2023_07_21/proposals_1_4000.json
next group file exists: proposals_out_2023_07_21/proposals_2_0.json
next file exists: proposals_out_2023_07_21/proposals_2_1000.json
next file exists: proposals_out_2023_07_21/proposals_2_2000.json
next file exists: proposals_out_2023_07_21/proposals_2_3000.json
next file exists: proposals_out_2023_07_21/proposals_2_4000.json
next group file exists: proposals_out_2023_07_21/proposals_3_0.json
next file exists: proposals_out_2023_07_21/proposals_3_1000.json
next file exists

got 937 proposals for 24 2000
